In [1]:
import os 
import IO
from dff_helper import process_dff_trials
from pathlib import Path


from roi_stage import run_roi_stage
from utils import group_trials_by_stimulus

In [7]:
%run motion.py

In [3]:
try:
    import tifffile as tiff
except Exception as e:
    raise ImportError("Please install 'tifffile' (pip install tifffile).") 



In [4]:
def load_tiff_stack(path: str | Path) -> np.ndarray:
    """Load a multi-page TIFF as (T, H, W)."""
    arr = tiff.imread(str(path))
    if arr.ndim == 2:
        arr = arr[None, ...]
    if arr.ndim != 3:
        raise ValueError(f"Expected 2D or 3D TIFF; got shape={arr.shape}")
    return arr

In [11]:
from pathlib import Path
import numpy as np

# --------------------------------------------------
# INPUT PATHS
# --------------------------------------------------

path_raw = Path(
    r"Z:\Individual_Folders\Tobi\WF_axonimaging\axonal_imaging_tobi\data\raw\JPCM-08699\250911_leica\22_42_interleaved\250911_112136"
)
path_processed = Path(
    r"Z:\Individual_Folders\Tobi\WF_axonimaging\axonal_imaging_tobi\data\processed\JPCM-08699\250911_leica\22_42_interleaved\250911_112136"
)
raw_movie_path = path_raw / "recording.tiff"          # or raw movie .npy
       # or raw movie 
output_dir = path_processed / "motion_correction"

# --------------------------------------------------
# LOAD MOVIE
# --------------------------------------------------

#movie = load_tiff_stack(raw_movie_path)   # shape: (T, H, W)
movie = np.load(raw_movie_path)   # shape: (T, H, W)

In [12]:
corrected_movie, motion_vectors, reference_image = apply_motion_correction(
    movie, None, None, "fourier", None, None
)

In [13]:
# --------------------------------------------------
# SAVE OUTPUTS
# --------------------------------------------------
output_dir.mkdir(parents=True, exist_ok=True)

save_motion_vectors(
    motion_vectors,
    output_dir / "motion_vectors_dff.npy",
)

save_corrected_recording(
    corrected_movie,
    output_dir / "movie_corrected_dff.npy",
)

save_reference_image(
    reference_image,
    output_dir / "reference_image_dff.png",
)

plot_motion_over_time(
    motion_vectors_raw= motion_vectors,
    movie_corrected= corrected_movie,
    reference_image= reference_image,
    output_path= output_dir
)

plot_stability_maps(
    movie, corrected_movie, output_dir
)

create_comparison_movie(
    movie, corrected_movie, output_dir / 'comparison_dff.mp4', 30
)

print("✅ Motion correction finished")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1144, 593) to (1152, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Motion correction finished


In [ ]:
fft_reference = fftn(reference_image)
motion_vectors_corrected = []

for frame_index in range(corrected_movie.shape[0]):
    shift = estimate_image_shift(
        fft_reference,
        fftn(corrected_movie[frame_index]),
        upsample_factor=20,
        space="fourier",
        return_error=False,
    )

    motion_vectors_corrected.append(shift)
motion_vectors_corrected = np.array(motion_vectors_corrected)

In [ ]:
plt.plot(motion_vectors_corrected[:,0])
plt.plot(motion_vectors_corrected[:,1])

In [ ]:
plt.plot(motion_vectors[:,0])
plt.plot(motion_vectors[:,1])